# extend_ApsimMet

### Loretta Clancy, 30-Oct- 2018

#### Takes an apsim formatted '.met' file and adds additional columns calculations so that it can be used for summarising information at a later time (when re-matched with the output from Apsim Simulations)


In [3]:
#Import the required libraries
import sys
import os
import datetime
import numpy as np
import pandas as pd
import math

#### Define the working directories and set some default variables

In [92]:
# define the working directories
metDir = "/OSM/CBR/AG_WHEATTEMP/work/ApsimNG-LC/met"



The following reads the directory specified and gets names of the '.met' files so that they can be processed.  

In [55]:
def generate_filelist(metfileDir):
    metfile_df = pd.DataFrame(columns=['fullname'])
    metfile_df.fullname = sorted(metDir+'/'+f for f in os.listdir(metfileDir) if f.endswith('.met'))
    #print(metfile_df.head())
    
    metfile_df['filename'] = metfile_df.apply(lambda x: os.path.basename(x['fullname']), axis=1)
    metfile_df['filename'] = metfile_df.apply(lambda x: os.path.splitext(x['filename'])[0], axis=1)
    metfile_df['filename'] = metfile_df["filename"].str.replace("silo_", "")
    metfile_df['longitude'] = metfile_df.apply(lambda x: x['filename'].split('-')[0], axis=1)
    metfile_df['longitude'] = metfile_df.apply(lambda x: float(x['longitude'])/100, axis=1)
    metfile_df['latitude'] = metfile_df.apply(lambda x: '-' + x['filename'].split('-')[1], axis=1)
    metfile_df['latitude'] = metfile_df.apply(lambda x: float(x['latitude'])/100, axis=1)
    metfile_df['filename'] = metfile_df.apply(lambda x: x['filename'] + ".met", axis=1)
    
    outfilename = metfileDir + "/" + "metList.csv"
    metfile_df.to_csv(outfilename, encoding='utf-8', index=False)

The following are some values that are required for calculation the Apsim Thermal Time.  

In [56]:
generate_filelist(metDir)

In [4]:
#these are required for calculations of thermal time
num3hr = int(24 / 3)
#print("num3hr: ", num3hr)    
t_range_fract = []

# pre calculate t_range_fract for speed reasons
for period in range(num3hr):
    calcValue = 0.92105 \
                + 0.1140 * period \
                - 0.0703 * math.pow(period, 2) \
                + 0.0053 * math.pow(period, 3)
    t_range_fract.append(calcValue)

#print("t_range_fract: ", t_range_fract)


#### Define the functions required to retrieve data

This section retrieves data from the database as well as the corresponding Apsim weather file.

In [5]:
def linint_3hrly_Temperature(tmax, tmin, xp, fp):
    '''
    Eight interpolations of the air temperature are calculated using 
    a three-hour correction factor.
    For each air three-hour air temperature, a value is calculated.
    The eight three-hour estimates are then averaged to obtain the daily value.
    '''

    #Local Variables
    tot = 0.0            #sum_of of 3 hr interpolations
    
    for period in range(1, num3hr):
        #get mean temperature for 3 hr period (oC)
        tmean_3hour = tmin + (t_range_fract[period-1] * (tmax - tmin))
        tot = tot + np.interp(tmean_3hour, xp,fp)
        #print("tmean_3hour: ", tmean_3hour, " - tot: ", tot)

    return tot / num3hr;


In [121]:
def process_ApsimWeather(filename, outfile, latitude):
    '''
    Reads an apsim weather ('.met') file, removes the header information,
    calculates and adds a date column (based on year and day), and the
    average temperature (based on maxt and mint).
    '''
    import math

    
    #these are required for calculations of thermal time
    num3hr = int(24 / 3)
    #print("num3hr: ", num3hr)    
    t_range_fract = []

    # pre calculate t_range_fract for speed reasons
    for period in range(num3hr):
        calcValue = 0.92105 \
                    + 0.1140 * period \
                    - 0.0703 * math.pow(period, 2) \
                    + 0.0053 * math.pow(period, 3)
        t_range_fract.append(calcValue)
    
    # these XYPairs are use when calculating Thermal Time 
    # and are specific to Wheat only
    xp = [0, 26, 37]
    fp = [0, 26, 0]
    
    #-------------
    # start working with the data here
    #-------------
    lineNo = 0
    with open(filename, "r") as f:
        for line in f:
            lineNo = lineNo + 1
            if line.startswith('year'):
                break;

    # return the data using the starting line no (determined above)
    # original column names=['year','day', 'radn', 'maxt', 'mint', 'rain', 'evap', 'vp']
    metData = pd.read_table(filename, sep='\s+', header=None, skiprows=lineNo+1, index_col=False,
                            names=['year','dayofYear', 'radiation', 'maxTemp', 'minTemp', 'rain', 'evap', 'vp'])

    # add the calculated columns
    metData['runDate'] = pd.to_datetime(metData['year'].astype(str) + " " + metData['dayofYear'].astype(str), format="%Y %j")
    
    #filter this file based on the dates we are working with
    #metData = metData[(metData['runDate'] >= startDate) & (metData['runDate'] <= endDate)] 
        
    # this may need to be the thermal time, not just average temp
    metData['avgTemp'] = (metData['maxTemp'] + metData['minTemp']) / 2

    # calculate the Apsim Thermal Time
    metData['ApsimTT'] = metData.apply(lambda x: linint_3hrly_Temperature(x['maxTemp'], x['minTemp'], xp, fp), axis=1)
    
    #convert the radiation from MJ/m2/day to Photosynthetically active radiation (PAR)
    metData['PARIO'] = metData['radiation'] * 0.47

    #convert the measurement unit for the radiation from MJ/m2/day to J/m2/day
    metData['radnJ'] = metData['radiation'] * 1000000

    metData['PQ'] = metData['PARIO'] / metData['avgTemp']

    # calculation the day length
    radians = math.pi/180
    lambdaRadians = float(latitude) * radians

    sinLAT = math.sin(lambdaRadians)
    cosLAT = math.cos(lambdaRadians)
    sinDMC = math.sin(radians * 23.45)

    #print("radians: ", radians)
    #print("lambdaRadians: ", lambdaRadians)
    #print("sinLAT: ", sinLAT)
    #print("cosLAT: ", cosLAT)
    #print("sinDMC: ", sinDMC)    
    
    metData['sinDEC'] = -sinDMC * np.cos(2 * math.pi * (metData['dayofYear'] + 10) / 365)
    metData['cosDEC'] = np.sqrt(1 - (metData['sinDEC'] * metData['sinDEC']))
    metData['a'] = sinLAT * metData['sinDEC']
    metData['b'] = cosLAT * metData['cosDEC']

    metData['daylength'] = 12 * (1 + (2 / math.pi) * np.arcsin(metData['a']/metData['b']))

    # calculate the Fraction Disfused Radiation (FDR)
    metData['hour'] = np.mod(metData['dayofYear'], 1) * 24
    metData['sinB'] = metData['a'] + metData['b'] * np.cos(2 * math.pi * (metData['hour'] - 12) / 24)
    metData['SC'] = 1367 * (1 + 0.033 * np.cos(2 * math.pi * (metData['dayofYear'] - 10) / 365))
    metData['sinINT'] = metData['a'] * metData['daylength'] + (24 * metData['b'] / math.pi) * \
                        np.cos((math.pi / 2) * ((metData['daylength'] / 12) - 1))

    metData['Ta'] = metData['radnJ'] / (metData['sinINT'] * 3600 * metData['SC'])
    metData['FDR'] = metData['Ta'] * -1.4545 + 1.2182

    # calculate the Evapotranspiration
    metData['vpsl'] = 238.102 * 17.32491 * ((metData['minTemp'] + metData['maxTemp']) /2) / \
                      (((metData['minTemp'] + metData['maxTemp']) / 2) + 238.102) ** 2
    metData['ETpt'] = 1.26 * (metData['radnJ']  * (metData['vpsl'] / (metData['vpsl'] + 0.067))) / 2454000

    #metData.round({'ApsimTT':4, 'PC':4, 'sinDEC':4, 'cosDEC':4, 'a':4, 'b':4, 'daylength':4})
    #metData.round({'hour':4, 'sinB':4, 'SC':4, 'sinINT':4, 'Ta':4, 'FDR':4, 'vpsl':4, 'ETpt':4})
    metData['ApsimTT'] = np.round(metData['ApsimTT'],decimals=5)
    metData['PARIO'] = np.round(metData['PARIO'],decimals=5)
    metData['PQ'] = np.round(metData['PQ'],decimals=5)
    metData['sinDEC'] = np.round(metData['sinDEC'],decimals=5)
    metData['cosDEC'] = np.round(metData['cosDEC'],decimals=5)
    metData['a'] = np.round(metData['a'],decimals=5)
    metData['b'] = np.round(metData['b'],decimals=5)
    metData['daylength'] = np.round(metData['daylength'],decimals=5)
    metData['hour'] = np.round(metData['hour'],decimals=5)
    metData['sinB'] = np.round(metData['sinB'],decimals=5)
    metData['SC'] = np.round(metData['SC'],decimals=5)
    metData['sinINT'] = np.round(metData['sinINT'],decimals=5)
    metData['Ta'] = np.round(metData['Ta'],decimals=5)
    metData['FDR'] = np.round(metData['FDR'],decimals=5)
    metData['vpsl'] = np.round(metData['vpsl'],decimals=5)
    metData['ETpt'] = np.round(metData['ETpt'],decimals=5)
    
    metData.to_csv(outfile, encoding='utf-8', index=False)

    #return metData
    #return metData
    

In [13]:
def getFileDetails(filename, row):
    
    metdf = pd.read_csv(filename)
    print(metdf.head())
    infile = metdf['fullname'].iloc[row-1]
    outfile = metdf['filename'].iloc[row-1]
    latitude = metdf['latitude'].iloc[row-1]
    
    return infile, outfile, latitude

## Process the data

The following processes a single database file, collating it with the corresponding weather data.  

##### NOTE:  dbname is defined at the top

In [57]:
filename = "/OSM/CBR/AG_WHEATTEMP/work/ApsimNG-LC/met/metList.csv"
getFileDetails(filename)
print(metdf.head())
testfile = metdf['fullname'].iloc[1]
outfile = metdf['filename'].iloc[1]
testlat = metdf['latitude'].iloc[1]
outfilename = outDir + "/" + outfile

                                            fullname        filename  \
0  /OSM/CBR/AG_WHEATTEMP/work/ApsimNG-LC/met/silo...  11400-2725.met   
1  /OSM/CBR/AG_WHEATTEMP/work/ApsimNG-LC/met/silo...  11405-2725.met   
2  /OSM/CBR/AG_WHEATTEMP/work/ApsimNG-LC/met/silo...  11405-2730.met   
3  /OSM/CBR/AG_WHEATTEMP/work/ApsimNG-LC/met/silo...  11405-2735.met   
4  /OSM/CBR/AG_WHEATTEMP/work/ApsimNG-LC/met/silo...  11410-2715.met   

   longitude  latitude  
0     114.00    -27.25  
1     114.05    -27.25  
2     114.05    -27.30  
3     114.05    -27.35  
4     114.10    -27.15  


In [14]:
filename = "/OSM/CBR/AG_WHEATTEMP/work/ApsimNG-LC/met/metList.csv"
#outDir = "/OSM/CBR/AG_WHEATTEMP/source/ApsimNG-LC/modifiedMet"
infile, outfile, latitude = getFileDetails(filename, 1)
#outfilename = outDir + "/" + outfile


                                            fullname        filename  \
0  /OSM/CBR/AG_WHEATTEMP/work/ApsimNG-LC/met/silo...  11400-2725.met   
1  /OSM/CBR/AG_WHEATTEMP/work/ApsimNG-LC/met/silo...  11405-2725.met   
2  /OSM/CBR/AG_WHEATTEMP/work/ApsimNG-LC/met/silo...  11405-2730.met   
3  /OSM/CBR/AG_WHEATTEMP/work/ApsimNG-LC/met/silo...  11405-2735.met   
4  /OSM/CBR/AG_WHEATTEMP/work/ApsimNG-LC/met/silo...  11410-2715.met   

   longitude  latitude  
0     114.00    -27.25  
1     114.05    -27.25  
2     114.05    -27.30  
3     114.05    -27.35  
4     114.10    -27.15  


In [15]:
print("infile: ", infile, " - outfile: ", outfile, " - latitude: ", latitude )

infile:  /OSM/CBR/AG_WHEATTEMP/work/ApsimNG-LC/met/silo_11400-2725.met  - outfile:  11400-2725.met  - latitude:  -27.25


In [113]:
data = process_ApsimWeather(testfile, outfilename, testlat)
data.head()

,year,dayofYear,radiation,maxTemp,minTemp,rain,evap,vp,runDate,avgTemp,...,b,daylength,hour,sinB,SC,sinINT,Ta,FDR,vpsl,ETpt
0,1957,1,27.0,27.0,17.5,0.0,9.6,13.0,1957-01-01,22.25,...,0.81831,13.68426,0,-0.63935,1411.57069,8.54892,0.62151,0.31422,1.35407,13.20947
1,1957,2,31.0,31.0,15.5,0.0,11.4,13.0,1957-01-02,23.25,...,0.81881,13.67730,0,-0.64048,1411.68391,8.54427,0.71392,0.17981,1.40412,15.19196
2,1957,3,31.0,33.0,15.0,0.0,11.8,13.0,1957-01-03,24.00,...,0.81936,13.66977,0,-0.64169,1411.78389,8.53920,0.71429,0.17927,1.44113,15.20975
3,1957,4,31.0,35.0,16.5,0.0,12.1,16.0,1957-01-04,25.75,...,0.81995,13.66165,0,-0.64301,1411.87060,8.53372,0.71470,0.17866,1.52577,15.24733
4,1957,5,30.0,32.5,18.0,0.0,11.0,18.0,1957-01-05,25.25,...,0.82058,13.65297,0,-0.64441,1411.94401,8.52783,0.69209,0.21155,1.50183,14.74559


0